In [2]:
from __future__ import division
import fasttext

## FastText

In [7]:
data_samples = open('trec_5500_train.','r').readlines()

In [27]:
data_labels = [l.split(' ')[0] for l in data_samples]
data_text = [l[len(label)+1:] for l,label in zip(data_samples, data_labels)]
class_labels = list(set(data_labels))
print 'There are', len(class_labels),'classes:'
print '\n'.join(class_labels)

There are 50 classes:
ENTY:animal
NUM:date
ENTY:event
ENTY:dismed
ABBR:abb
ENTY:product
LOC:mount
ENTY:body
ENTY:techmeth
ENTY:instru
NUM:ord
LOC:city
ENTY:color
ENTY:food
NUM:weight
LOC:other
NUM:dist
ENTY:plant
ENTY:currency
ENTY:termeq
NUM:perc
NUM:code
LOC:state
ABBR:exp
HUM:gr
ENTY:word
DESC:reason
ENTY:symbol
HUM:desc
ENTY:letter
NUM:period
NUM:other
ENTY:other
ENTY:cremat
ENTY:veh
NUM:count
LOC:country
ENTY:religion
DESC:manner
HUM:ind
NUM:speed
ENTY:sport
ENTY:substance
ENTY:lang
NUM:temp
DESC:desc
DESC:def
NUM:money
NUM:volsize
HUM:title


In [9]:
ft_class_prefix = '__label__'
ft_data_samples = ['%s%s'%(ft_class_prefix, s) for s in data_samples]

In [10]:
save_file = open('train_data.txt', 'w')
save_file.write(''.join(ft_data_samples))
save_file.close()

In [11]:
clf = fasttext.supervised('train_data.txt', 'question.clf', label_prefix=ft_class_prefix)

In [12]:
test_file = open('test_data.txt', 'w')
test_file.write(''.join(data_text))
test_file.close()

In [13]:
test_file = open('test_data.txt')
test_samples = test_file.readlines()
output = clf.predict(test_samples)
pred_label = [x[0] for x in output]

In [14]:
match_count = [1 if x==y else 0 for x,y in zip(data_labels, pred_label)]
precision = sum(match_count) / len(output)
print precision

0.468268525312


## TREC Eval

In [257]:
def read_trec_data(data_path):
    samples = open(data_path,'r').readlines()
    Y = [l.split(' ')[0] for l in samples]
    X = [l[len(y)+1:] for l,y in zip(samples, Y)]
    return X,Y

In [252]:
def format_trec_to_ft(data_path, save_path):
    samples = open(data_path,'r').readlines()
    ft_class_prefix = '__label__'
    ft_data_samples = ['%s%s'%(ft_class_prefix, s) for s in samples]
    save_file = open(save_path, 'w')
    save_file.write(''.join(ft_data_samples))
    save_file.close()

In [290]:
def test_model(model, X, Y):
    output = clf.predict(X)
    pred_label = [x[0] for x in output]
    match_count = [1 if y_hat==y else 0 for y_hat,y in zip(pred_label, Y)]
    precision = sum(match_count) / len(output)
    print precision
    return precision

In [267]:
format_trec_to_ft('trec_5500_train.txt', 'train_data.txt')

In [268]:
clf = fasttext.supervised('train_data.txt', 'question.clf', label_prefix=ft_class_prefix)

In [291]:
X,Y = read_trec_data('trec_5500_test.txt')
test_model(clf, X, Y)

0.526


0.526

### Testing 10-fold

In [292]:
from sklearn.model_selection import KFold
kf = KFold(n_splits=10)
kf.get_n_splits(data_text)
k = 0
P = []
for train_index, test_index in kf.split(data_text):
    save_file = open('tmp.txt', 'w')
    save_file.write(''.join([ft_data_samples[idx] for idx in train_index]))
    save_file.close()
    clf = fasttext.supervised('tmp.txt', 'tmp.clf', label_prefix=ft_class_prefix)
    
    output = clf.predict([data_text[idx] for idx in test_index])
    pred_label = [x[0] for x in output]
    match_count = [1 if x.split(':')[0]==y.split(':')[0] else 0 for x,y in zip([data_labels[idx] for idx in test_index], pred_label)]
    precision = sum(match_count) / len(output)
    k += 1
    print 'Fold',k,':',precision
    P.append(precision)

print 'mAP: ', sum(P)/len(P)

Fold 1 : 0.553113553114
Fold 2 : 0.529304029304
Fold 3 : 0.521100917431
Fold 4 : 0.500917431193
Fold 5 : 0.530275229358
Fold 6 : 0.566972477064
Fold 7 : 0.521100917431
Fold 8 : 0.517431192661
Fold 9 : 0.563302752294
Fold 10 : 0.537614678899
mAP:  0.534113317875


## External test

In [31]:
q = 'show me pictures back when I was a quarterback'
answer = clf.predict_proba([q])
print answer

[[(u'NUM:date', 0.152344)]]
